In [83]:
import pandas as pd
import numpy as np
import math

In [9]:
# NFL Import
from espn_api.football import *

In [16]:
espn_s2 = "{DE08842B-7384-4C68-B7C5-AB3BB102E995}"
swid = "AEBDU030JkQcU9Pa5FCCQczqZoZIGjrRqt63%2BY8iYA3Eb2CE4KbLx0xdfwG2EFFJPA%2FrpUSGfuOVRfCCeX%2F%2BYX%2BBmJ6gFvTvE7UMZpSgH0WBgT%2BZMWYVWC6cJARuctk5ck5uToViuedkNwdMX55Qbx9uaMfknXGUsd5YtNXpeDnzYeVEj6x3Hrrfhi0E8LJCkBVBulS3LssLdUc2Bh057ZnVIVNqHN25DLcGnGtvCZE7js8lug0cj7qoTklTMJKvvn6w9WE1iWg0gKod40RpuorelZBhPXagJfz3n5h088%2BkjqHRIzbAdsurOux00ys3V1c%3D"

In [43]:
league = League(league_id=605278, year=2024, espn_s2=espn_s2, swid=swid)

In [44]:
print(league.teams)

[Team(Jack FC), Team(The Abusement Park), Team(Ricardo 69), Team(Undisputed King), Team(Big Chief), Team(Night Dragon apwoeidjxcjsmwoxof), Team(Weeklong Marlin), Team(Shivakamini), Team(buffalo vanilla ice ), Team(Chang Soda), Team(Cole's Cool Team), Team(Joe 🥶ow)]


In [116]:
league.teams[0].team_name

'Jack FC'

In [58]:
box_scores = league.box_scores(1)

In [59]:
box_scores

[Box Score(Team(Jack FC) at Team(Big Chief)),
 Box Score(Team(buffalo vanilla ice ) at Team(Ricardo 69)),
 Box Score(Team(Cole's Cool Team) at Team(Weeklong Marlin)),
 Box Score(Team(The Abusement Park) at Team(Undisputed King)),
 Box Score(Team(Chang Soda) at Team(Joe 🥶ow)),
 Box Score(Team(Shivakamini) at Team(Night Dragon apwoeidjxcjsmwoxof))]

In [60]:
print(f"Matchup 0 is {box_scores[0].home_team} vs {box_scores[0].away_team}")
print(f"This score of Matchup 0 was {box_scores[0].home_score} : {box_scores[0].away_score}")

Matchup 0 is Team(Big Chief) vs Team(Jack FC)
This score of Matchup 0 was 91.46 : 105.9


In [79]:
def spread(week):
    box_scores = league.box_scores(week)

    spread_df = pd.DataFrame(columns=["home", "away", "spread"])

    for matchup in range(6):
        home_team = box_scores[matchup].home_team
        away_team = box_scores[matchup].away_team

        home_proj = box_scores[matchup].home_projected
        away_proj = box_scores[matchup].away_projected

        diff = round((home_proj - away_proj), 2)

        if diff > 0: spread = f"-{diff}"
        else: spread = f"+{-1*diff}"

        new_row = {"home": home_team, "away": away_team, "spread": spread}
        spread_df = spread_df._append(new_row, ignore_index=True)
    
    return spread_df

In [80]:
df_test = spread(11)
df_test

,home,away,spread
0,Team(The Abusement Park),Team(Ricardo 69),+15.69
1,Team(Undisputed King),Team(Jack FC),-6.64
2,Team(Chang Soda),Team(Big Chief),+4.17
3,Team(Shivakamini),Team(buffalo vanilla ice ),-5.17
4,Team(Night Dragon apwoeidjxcjsmwoxof),Team(Cole's Cool Team),-2.72
5,Team(Joe 🥶ow),Team(Weeklong Marlin),-1.67


In [173]:
from ffbookie import FFBookie

In [183]:
def calculate_money_lines(home_proj, away_proj, k=0.05, vig=0.048):
    # Calculate the spread
    spread = home_proj - away_proj

    # Convert spread to win probabilities using a logistic function
    home_win_prob = 1 / (1 + math.exp(-k * spread))
    away_win_prob = 1 - home_win_prob

    # Adjust probabilities to include vig
    total_prob = home_win_prob + away_win_prob
    home_win_prob_adjusted = home_win_prob / total_prob * (1 - vig)
    away_win_prob_adjusted = away_win_prob / total_prob * (1 - vig)

    print(f"{home_win_prob_adjusted}, {away_win_prob_adjusted}")

    # Convert probabilities to money lines
    def prob_to_money_line(prob):
        if prob > 0.5:  # Favorite
            return round(-100 / (abs(prob - 1)))
        else:  # Underdog
            return round((1 / prob - 1) * 100)

    home_money_line = prob_to_money_line(home_win_prob_adjusted)
    away_money_line = prob_to_money_line(away_win_prob_adjusted)

    if home_money_line > 0 and away_money_line > 0:
        home_money_line *= -1
        away_money_line *= -1

    return home_money_line, away_money_line

In [186]:
home_proj = 110
away_proj = 115

home_ml, away_ml = calculate_money_lines(home_proj, away_proj, k=0.05, vig=0.048)
print(f"Home Money Line: {home_ml}")
print(f"Away Money Line: {away_ml}")

0.4168079711567202, 0.5351920288432798
Home Money Line: 140
Away Money Line: -215


In [181]:
bookie = FFBookie()

week = 13
bookie.calculate_odds(week)

bookie.odds_df

,home,away,spread (+110),O/U (+110),ML
0,Big Chief,The Abusement Park,-7.4,212.4,"home ML: -229, away ML: +157"
1,buffalo vanilla ice,Undisputed King,-6.39,196.1,"home ML: -223, away ML: +150"
2,Cole's Cool Team,Chang Soda,+27.97,203.0,"home ML: +430, away ML: -423"
3,Weeklong Marlin,Shivakamini,+0.2,227.4,"home ML: +111, away ML: 109"
4,Ricardo 69,Night Dragon apwoeidjxcjsmwoxof,+0.06,224.9,"home ML: +110, away ML: 110"
5,Jack FC,Joe 🥶ow,-1.63,221.7,"home ML: +102, away ML: 119"
